# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [13]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='w'].call),sum(data[data.race=='b'].call))
print(sum(data.call))

235.0 157.0
392.0


In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [51]:
w = data[data.race=='w'].call
b = data[data.race=='b'].call


In [58]:
#test = data[data.yearsexp==22][['call','race','sex','education']]
#print(test)

In [60]:
#list(data.columns.values)
#get column titles

In [7]:
# Your solution to Q3 here

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

# 1. 

In [ ]:
#we can use two sample z test. Since the data >30, it is large enough to approximate the population. 
# CLT applies to this situation as we want to use samples to make conclusions about the population.

# 2. 

In [ ]:
#H0: Mean(data[data.race=='w'].call) == Mean(data[data.race=='b'].call)
#Ha: Mean(data[data.race=='w'].call) <> Mean(data[data.race=='b'].call)

# 3.

In [30]:
#bootstrap
np.random.seed(30)
def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data,func)

    return bs_replicates
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

mean_call = np.mean(data.call)
shifted_w = w - np.mean(w) + mean_call
shifted_b = b - np.mean(b) + mean_call

bs_replicates_w = draw_bs_reps(shifted_w,np.mean,size = 10000)
bs_replicates_b = draw_bs_reps(shifted_b,np.mean,size = 10000)

bs_replicates = bs_replicates_w - bs_replicates_b
data_mean_diff = np.mean(w)-np.mean(b)

p = np.sum(bs_replicates >= data_mean_diff)/len(bs_replicates)
ci = np.percentile(bs_replicates, [2.5,97.5])
margin_of_error = np.mean(bs_replicates) - ci[0]
print(format(p, '.4f'))
print(ci)
print(margin_of_error)

0.0000
[-0.01519509  0.01519505]
0.015264145087078214


In [21]:
print(data_mean_diff)

0.03203285485506058


In [22]:
bs_replicates

array([ 0.00944556,  0.00082133,  0.00657082, ..., -0.00821358,
       -0.0094456 , -0.00205342])

In [44]:
#z test
z_score = data_mean_diff/np.sqrt(np.var(w)/len(w)+np.var(b)/len(b))
p = 2*(1-stats.norm.cdf(z_score))
z_critical = stats.norm.ppf(q = 0.975) 
z_margin_of_error = z_critical * np.sqrt(np.var(w)/len(w)+np.var(b)/len(b))
ci=(data_mean_diff  - z_margin_of_error,data_mean_diff  + z_margin_of_error)
print(p)
print(ci)
print(z_margin_of_error)

3.862011079980299e-05
(0.016777851170209572, 0.04728785853991158)
0.015255003684851004


# 4.

based on the low p value, we reject null hypothesis. This means that there is statistical significance in the difference in call rate between white and black sounding names

# 5. 

It does not mean race/name is the most important factor in call back success. It is a source of bias given identical resumes but other factors that are not consistent in actual resume submissions such as work experience and education can have bigger impact on call back success. 

To expand on the analysis, I can hold race/name consistent and test the effect of in number of years of education and number of years of work experience. 